# Profiling Julia code

In [39]:
using Base.Profile

In [52]:
function myfunc()
           A = rand(200, 200, 400)
           maximum(A)
       end

myfunc()

0.9999998402526629

In [62]:
Profile.clear()

In [63]:
@profile myfunc()

0.9999999997074176

In [68]:
Profile.print(format=:flat)

 Count File                        Line Function                               
    56 ./<missing>                   -1 anonymous                              
    26 ./In[52]                       2 myfunc()                               
    30 ./In[52]                       3 myfunc()                               
    24 ./dSFMT.jl                    84 dsfmt_fill_array_close_open!(::Base... 
    56 ./loading.jl                 515 include_string(::String, ::String)     
    56 ./profile.jl                  23 macro expansion                        
    24 ./random.jl                  433 rand!(::MersenneTwister, ::Array{Fl... 
     2 ./random.jl                  279 rand                                   
     2 ./random.jl                  280 rand                                   
     2 ./random.jl                  368 rand                                   
     2 ./random.jl                  371 rand                                   
    30 ./reduce.jl                  280 

In [66]:
#Profile.print()

### More examples

In [1]:
using Base.Profile

In [2]:
function examplefunc(x)
    x = x + 2
    x = 2 * x
    return x
end

examplefunc (generic function with 1 method)

In [3]:
aux = rand(10000);

In [4]:
Profile.clear()

In [14]:
@time examplefunc(aux);

  0.000102 seconds (8 allocations: 156.563 KiB)


In [15]:
@profile examplefunc(aux);

In [18]:
Profile.print()

running it multiple times), or adjust the delay between samples with
Profile.init().


In [19]:
@profile (for i = 1:100; examplefunc(aux); end)

In [20]:
Profile.print()

6 ./task.jl:335; (::IJulia.##14#17)()
 6 ...IJulia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  5 ...src/execute_request.jl:154; execute_request(::ZMQ.Socket, ::I...
   5 .../Compat/src/Compat.jl:478; include_string(::Module, ::String...
    5 ./loading.jl:515; include_string(::String, ::String)
     5 ./<missing>:?; anonymous
      5 ./profile.jl:23; macro expansion
       5 ./In[19]:1; macro expansion
        4 ./In[2]:2; examplefunc
         4 ./arraymath.jl:47; +(::Array{Float64,1}, ::Int64)
          4 ./broadcast.jl:434; broadcast
           4 ./broadcast.jl:314; broadcast_c
            4 ./broadcast.jl:268; broadcast_t
             4 ./broadcast.jl:139; _broadcast!
              4 ./broadcast.jl:147; macro expansion
               4 ./simdloop.jl:73; macro expansion
                1 ./broadcast.jl:151; macro expansion
                3 ./broadcast.jl:153; macro expansion
        1 ./In[2]:3; examplefunc
         1 ./arraymath.jl:44; *(::Int64, ::Array{Float64,1})
          1 .

In [21]:
Profile.print(C=true)

6 ..._9-x64/build/src/task.c:267; start_task
 6 ...64/build/src/./julia.h:1424; jl_apply
   6 ./task.jl:335; (::IJulia.##14#17)()
    6 ...Julia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
     6 .../build/src/builtins.c:464; jl_f__apply_latest
      6 ...build/src/builtins.c:426; jl_f__apply
       6 ...build/src/./julia.h:1424; jl_apply
         5 .../execute_request.jl:154; execute_request(::ZMQ.Socket,...
           5 ...mpat/src/Compat.jl:478; include_string(::Module, ::S...
            5 ...ild/src/builtins.c:496; jl_toplevel_eval_in
             5 ...ld/src/toplevel.c:577; jl_toplevel_eval_flex
              5 ...src/interpreter.c:34; jl_interpret_toplevel_expr
               5 ...src/interpreter.c:242; eval
                5 ...src/interpreter.c:75; do_call
                 5 ...b/julia/sys.dylib:?; jlcall_include_string_26781
                  5 ./loading.jl:515; include_string(::String,...
                   5 ...build/src/ast.c:873; jl_parse_eval_all
                    5 ..

#### example 2

In [136]:
Profile.clear()

In [137]:
function testfunc()
    x = rand(10000, 1000)
    y = std(x, 1)
    return y
end

testfunc (generic function with 1 method)

In [138]:
@profile testfunc()

1×1000 Array{Float64,2}:
 0.287737  0.289342  0.287736  0.290109  …  0.288354  0.290238  0.288198

In [23]:
Profile.print()

6 ./task.jl:335; (::IJulia.##14#17)()
 6 ...IJulia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  5 ...src/execute_request.jl:154; execute_request(::ZMQ.Socket, ::I...
   5 .../Compat/src/Compat.jl:478; include_string(::Module, ::String...
    5 ./loading.jl:515; include_string(::String, ::String)
     5 ./<missing>:?; anonymous
      5 ./profile.jl:23; macro expansion
       5 ./In[19]:1; macro expansion
        4 ./In[2]:2; examplefunc
         4 ./arraymath.jl:47; +(::Array{Float64,1}, ::Int64)
          4 ./broadcast.jl:434; broadcast
           4 ./broadcast.jl:314; broadcast_c
            4 ./broadcast.jl:268; broadcast_t
             4 ./broadcast.jl:139; _broadcast!
              4 ./broadcast.jl:147; macro expansion
               4 ./simdloop.jl:73; macro expansion
                1 ./broadcast.jl:151; macro expansion
                3 ./broadcast.jl:153; macro expansion
        1 ./In[2]:3; examplefunc
         1 ./arraymath.jl:44; *(::Int64, ::Array{Float64,1})
          1 .

### ProfileView

In [26]:
using ProfileView

INFO: Precompiling module ProfileView.
INFO: Precompiling module ProfileViewSVG.


In [27]:
function profile_test(n)
    for i = 1:n
        A = randn(100,100,20)
        m = maximum(A)
        Afft = fft(A)
        Am = mapslices(sum, A, 2)
        B = A[:,:,5]
        Bsort = mapslices(sort, B, 1)
        b = rand(100)
        C = B.*b
    end
end

profile_test (generic function with 1 method)

In [28]:
profile_test(1)  # run once to trigger compilation
Profile.clear()  # in case we have any previous profiling data
@profile  profile_test(100)

In [33]:
#ProfileView.view()

In [32]:
Base.Profile.

Base.Profile